In [1]:
import os

In [2]:
%pwd

'a:\\mlflow\\Heart-Failure-Prediction-with-Machine-Learning-MLflow\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'a:\\mlflow\\Heart-Failure-Prediction-with-Machine-Learning-MLflow'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str 
    local_data_file: Path 
    unzip_dir: Path 

In [8]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

ModuleNotFoundError: No module named 'ml_project'

In [22]:
class ConfigrationManger:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH,    
        schema_file_path = SCHEMA_FILE_PATH,
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root_dir])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )
        return data_ingestion_config
        
    



In [14]:
from urllib import request
import zipfile
from ml_project.utils.common import get_size
from ml_project import logger

In [ ]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):

        if not os.path.exists(self.config.local_data_file):
            file_name, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )

            logger.info(f"{file_name} downloaded with following info: {headers}")
            logger.info(f"File size: {get_size(Path(self.config.local_data_file))}")

        else:
            logger.info(f"File {self.config.local_data_file} already exists.")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

        


In [29]:
try:
    config_manager = ConfigrationManger()
    data_ingestion_config = config_manager.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    print(e)
    raise e

[2025-07-17 16:59:38,133 : INFO: yaml file: config\config.yaml loaded successfully]
[2025-07-17 16:59:38,136 : INFO: yaml file: params.yaml loaded successfully]
[2025-07-17 16:59:38,136 : INFO: yaml file: schema.yaml loaded successfully]
[2025-07-17 16:59:38,136 : INFO: created directory at: artifacts]
[2025-07-17 16:59:38,136 : INFO: created directory at: artifacts/data_ingestion]
[2025-07-17 16:59:38,646 : INFO: artifacts\data_ingestion\data.zip downloaded with following info: Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: FC9C:3873BC:8FB4EA:B601B3:68796439
Accept-Ranges: bytes
Date: Thu, 17 Jul 2025 20:59:38 GMT
Via: 1.1 varnish
X-Serv